In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


This script is used to estimate the number of features with strong regulatory action using SCRaPL's posterior in brain data.

In [ ]:
from IPython import display
import pandas as pd
import numpy as np
import numpy as np
import numpy.ma as ma
import scipy
import scipy.stats
from scipy.stats import gaussian_kde
from scipy.stats import t

from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from matplotlib import cm
from matplotlib.colors import Normalize 
from matplotlib.offsetbox import AnchoredText

from tensorflow import keras
from sklearn.neighbors import KernelDensity

from tensorflow.keras import layers
import tensorflow_probability as tfp
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

import pickle
from timeit import default_timer as timer
Folder = '/content/drive/MyDrive/'

Load posterior latent correlations for original and negative control data.

In [ ]:
neg = 0
with open(Folder+'SCRaPL/Real/Results_atac_col/nuts_cor_atac.pickle', 'rb') as handle:
#with open(Folder+'SCRaPL/Real/Results_atac_neg_col/nuts_cor_atac_neg'+str(neg)+'.pickle', 'rb') as handle:
    cor_nuts = pickle.load(handle)

In [ ]:
def integral(y, x):
    dx = (x[:,-1] - x[:,0]) / (int(x.shape[1]) - 1)
    return  tf.multiply((y[:,0] + y[:,-1])/2+tf.reduce_sum(y[:,1:-1],axis=1) , dx)

In [ ]:
from sklearn.neighbors import KernelDensity

cor_exmp = cor_nuts.numpy()
x_d = np.linspace(-0.9, 3, 1000)

xx= np.apply_along_axis(lambda x: KernelDensity(bandwidth=0.1, kernel='gaussian').fit(x[:,None]),0, cor_exmp)
gam = 0.145

In [ ]:
p_bay  = np.zeros(4249)
for ii in range(p_bay.shape[0]):
    pp =  np.exp(xx[ii].score_samples(x_d[:,None]))
    p_bay[ii] =  integral(pp[None,np.abs(x_d)<2*np.math.atanh(gam)], x_d[None,np.abs(x_d)<2*np.math.atanh(gam)])
    if np.mod(ii,200)==0:
      print(ii)

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200


Estimate features and EFDR for original and negative control data

In [ ]:
alpha = 0.71
pp1 = 1-p_bay

zz1 = tf.greater(pp1,alpha)
zz2 = tf.less_equal(pp1,alpha)  

EFDR = tf.reduce_mean(tf.gather(p_bay,tf.where(zz1==True)))

num_features = tf.shape(tf.where(zz1==True))[0]
feature_ind = tf.cast(zz1,dtype=tf.int16)

In [ ]:
#0 --> 145/710 --> 4180
#1 --> 145/770 --> 31
#2 --> 145/790 --> 22
#3 --> 145/785 --> 27
#4 --> 145/830 --> 10
#5 --> 145/800 --> 21

In [ ]:
#np.savetxt(Folder+'SCRaPL/Real/Paper_Analysis/SCRaPL_meta/brain_EFDR_neg'+str(neg)+".csv", EFDR.numpy()[np.newaxis,np.newaxis], delimiter=",")
#np.savetxt(Folder+'SCRaPL/Real/Paper_Analysis/SCRaPL_meta/brain_feature_ind_neg'+str(neg)+".csv", feature_ind.numpy() , delimiter=",")
#np.savetxt(Folder+'SCRaPL/Real/Paper_Analysis/SCRaPL_meta/brain_p_bay_neg'+str(neg)+".csv", p_bay , delimiter=",")